In [68]:
import numpy as np
import pandas as pd

In [69]:
df = pd.read_csv("A_year_of_pizza_sales_from_a_pizza_place_872_68.csv")

                

In [70]:
df

,Unnamed: 0,X,id,date,time,name,size,type,price
0,1,1,2015-000001,2015-01-01,11:38:36,hawaiian,M,classic,13.25
1,2,2,2015-000002,2015-01-01,11:57:40,classic_dlx,M,classic,16.00
2,3,3,2015-000002,2015-01-01,11:57:40,mexicana,M,veggie,16.00
3,4,4,2015-000002,2015-01-01,11:57:40,thai_ckn,L,chicken,20.75
4,5,5,2015-000002,2015-01-01,11:57:40,five_cheese,L,veggie,18.50
...,...,...,...,...,...,...,...,...,...
49569,49570,49570,2015-021348,2015-12-31,21:23:10,four_cheese,L,veggie,17.95
49570,49571,49571,2015-021348,2015-12-31,21:23:10,napolitana,S,classic,12.00
49571,49572,49572,2015-021348,2015-12-31,21:23:10,ckn_alfredo,M,chicken,16.75
49572,49573,49573,2015-021349,2015-12-31,22:09:54,mexicana,L,veggie,20.25


In [71]:
df.isnull().sum()

Unnamed: 0    0
X             0
id            0
date          0
time          0
name          0
size          0
type          0
price         0
dtype: int64

In [72]:
df = df.drop( columns = ["Unnamed: 0", "X"])

In [73]:
df

,id,date,time,name,size,type,price
0,2015-000001,2015-01-01,11:38:36,hawaiian,M,classic,13.25
1,2015-000002,2015-01-01,11:57:40,classic_dlx,M,classic,16.00
2,2015-000002,2015-01-01,11:57:40,mexicana,M,veggie,16.00
3,2015-000002,2015-01-01,11:57:40,thai_ckn,L,chicken,20.75
4,2015-000002,2015-01-01,11:57:40,five_cheese,L,veggie,18.50
...,...,...,...,...,...,...,...
49569,2015-021348,2015-12-31,21:23:10,four_cheese,L,veggie,17.95
49570,2015-021348,2015-12-31,21:23:10,napolitana,S,classic,12.00
49571,2015-021348,2015-12-31,21:23:10,ckn_alfredo,M,chicken,16.75
49572,2015-021349,2015-12-31,22:09:54,mexicana,L,veggie,20.25


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49574 entries, 0 to 49573
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49574 non-null  object 
 1   date    49574 non-null  object 
 2   time    49574 non-null  object 
 3   name    49574 non-null  object 
 4   size    49574 non-null  object 
 5   type    49574 non-null  object 
 6   price   49574 non-null  float64
dtypes: float64(1), object(6)
memory usage: 2.6+ MB


In [75]:
df["name"].value_counts()

name
classic_dlx     2453
bbq_ckn         2432
hawaiian        2422
pepperoni       2418
thai_ckn        2371
cali_ckn        2370
sicilian        1938
spicy_ital      1924
southw_ckn      1917
big_meat        1914
four_cheese     1902
ital_supr       1884
veggie_veg      1526
mexicana        1484
napolitana      1464
prsc_argla      1457
spinach_fet     1446
peppr_salami    1446
ital_cpcllo     1438
the_greek       1420
five_cheese     1409
pep_msh_pep     1359
green_garden     997
ckn_alfredo      987
ital_veggie      981
ckn_pesto        973
spin_pesto       970
soppressata      961
spinach_supr     950
calabrese        937
mediterraneo     934
brie_carre       490
Name: count, dtype: int64

In [76]:
df["size"].unique()

array(['M', 'L', 'S', 'XL', 'XXL'], dtype=object)

In [77]:
df["type"].unique()

array(['classic', 'veggie', 'chicken', 'supreme'], dtype=object)

In [78]:
df_price = df["price"].value_counts()
df_price.sort_index(ascending = False)

price
35.95      28
25.50     552
23.65     490
21.00     190
20.75    8891
20.50    2026
20.25    3093
18.50    1409
17.95    1316
17.50     384
16.75    4380
16.50    4111
16.25    1136
16.00    4522
15.25     728
14.75     586
14.50     397
13.25     483
12.75    2529
12.50    3380
12.25     850
12.00    5744
11.00     578
10.50    1020
9.75      751
Name: count, dtype: int64

In [79]:
df["time"] = pd.to_datetime(df['date'] + ' ' + df['time'])

In [80]:
df

,id,date,time,name,size,type,price
0,2015-000001,2015-01-01,2015-01-01 11:38:36,hawaiian,M,classic,13.25
1,2015-000002,2015-01-01,2015-01-01 11:57:40,classic_dlx,M,classic,16.00
2,2015-000002,2015-01-01,2015-01-01 11:57:40,mexicana,M,veggie,16.00
3,2015-000002,2015-01-01,2015-01-01 11:57:40,thai_ckn,L,chicken,20.75
4,2015-000002,2015-01-01,2015-01-01 11:57:40,five_cheese,L,veggie,18.50
...,...,...,...,...,...,...,...
49569,2015-021348,2015-12-31,2015-12-31 21:23:10,four_cheese,L,veggie,17.95
49570,2015-021348,2015-12-31,2015-12-31 21:23:10,napolitana,S,classic,12.00
49571,2015-021348,2015-12-31,2015-12-31 21:23:10,ckn_alfredo,M,chicken,16.75
49572,2015-021349,2015-12-31,2015-12-31 22:09:54,mexicana,L,veggie,20.25


In [81]:
df = df.drop(columns = ["date"])

In [82]:
df

,id,time,name,size,type,price
0,2015-000001,2015-01-01 11:38:36,hawaiian,M,classic,13.25
1,2015-000002,2015-01-01 11:57:40,classic_dlx,M,classic,16.00
2,2015-000002,2015-01-01 11:57:40,mexicana,M,veggie,16.00
3,2015-000002,2015-01-01 11:57:40,thai_ckn,L,chicken,20.75
4,2015-000002,2015-01-01 11:57:40,five_cheese,L,veggie,18.50
...,...,...,...,...,...,...
49569,2015-021348,2015-12-31 21:23:10,four_cheese,L,veggie,17.95
49570,2015-021348,2015-12-31 21:23:10,napolitana,S,classic,12.00
49571,2015-021348,2015-12-31 21:23:10,ckn_alfredo,M,chicken,16.75
49572,2015-021349,2015-12-31 22:09:54,mexicana,L,veggie,20.25


In [83]:
df["weekday nº"] = df['time'].dt.weekday

In [84]:
df

,id,time,name,size,type,price,weekday nº
0,2015-000001,2015-01-01 11:38:36,hawaiian,M,classic,13.25,3
1,2015-000002,2015-01-01 11:57:40,classic_dlx,M,classic,16.00,3
2,2015-000002,2015-01-01 11:57:40,mexicana,M,veggie,16.00,3
3,2015-000002,2015-01-01 11:57:40,thai_ckn,L,chicken,20.75,3
4,2015-000002,2015-01-01 11:57:40,five_cheese,L,veggie,18.50,3
...,...,...,...,...,...,...,...
49569,2015-021348,2015-12-31 21:23:10,four_cheese,L,veggie,17.95,3
49570,2015-021348,2015-12-31 21:23:10,napolitana,S,classic,12.00,3
49571,2015-021348,2015-12-31 21:23:10,ckn_alfredo,M,chicken,16.75,3
49572,2015-021349,2015-12-31 22:09:54,mexicana,L,veggie,20.25,3
